In [1]:
#Import libraries
import numpy as np
import math as math
import cmath as cmath
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si


In [2]:
K= [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
T = [0.1, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.0]

In [3]:
n_samples = 1
v0 = np.random.uniform(low = 0.0001, high=0.04, size=(n_samples))
rho = np.random.uniform(low = -0.95, high=-0.1, size=( n_samples))
sigma = np.random.uniform(low = 0.01, high= 1, size=(n_samples))
theta = np.random.uniform(low = 0.01, high= 0.2, size=(n_samples))
kappa = np.random.uniform(low = 1, high = 10 , size=(n_samples))

In [4]:
params_volsurface_map = {}
for i in range(n_samples):
    vol_surface = pd.DataFrame()
    for strike in K:
        for maturity in T:
            hc=Heston(S0=1,K=strike,tau=maturity, r=0.0, kappa=kappa[i],theta=theta[i],v0=v0[i],lamda=0,sigma=sigma[i],rho=rho[i]);
            price=hc.price(0.00001,1,10000);

            c = price['Call price']

            #underlying, strike, maturity, r, initial_guess
            iv = newton_vol_call(1,strike,maturity, c , 0, 0.25)
            vol_surface = vol_surface.append(pd.DataFrame([[iv, strike, maturity]], columns = ['IV', 'Strike', 'Maturity']), ignore_index=True)
            #print(pd.DataFrame([[iv, strike, maturity]], columns = ['IV', 'Strike', 'Maturity']))
    params_volsurface_map[i] = vol_surface

NameError: name 'Heston' is not defined

In [6]:
#Class of Heston Stochastic Volatility Model
#Using Euler Discretization and Milstein Discretization
import numpy as np
from numpy.random import standard_normal

class HestonModel: 
    def __init__(self, nSteps, nPaths, s0, v0, r,theta, kappa, lamda, rho):
        self._nSteps = nSteps
        self._nPaths = nPaths
        #Initial stock price
        self._s0 = s0
        #Initial volatility
        self._v0 = v0
        #risk free rate
        self._r = r
        #long term volatility(equiribrium level)
        self._theta = theta
        #Mean reversion speed of volatility
        self._kappa = kappa
        #lambda(volatility of Volatility)
        self._lamda = lamda
        #rho
        self._rho = rho
    #Euler Discretization
    def _generate_path(self, dt):
        s = np.zeros(self._nSteps + 1)
        v = np.zeros(self._nSteps + 1)
        s[0] = self._s0            
        v[0] = self._v0
        dW1 = standard_normal(self._nSteps)
        dW2 = self._rho * dW1 + (1 - self._rho**2)**(0.5) * standard_normal(self._nSteps)
        for j in range(0, self._nSteps):
            s[j + 1] = s[j] * np.exp((self._r - 0.5 * v[j]) * dt + (v[j] * dt)**(0.5) * dW1[j])
            v[j + 1] = max(v[j] + (self._kappa * (self._theta - v[j]) * dt) + self._lamda * (v[j] * dt)**(0.5) * dW2[j], 0)
        return s
    #Milstein Discretization
    def _generate_path_Mil(self, dt):
        s = np.zeros(self._nSteps + 1)
        v = np.zeros(self._nSteps + 1)
        s[0] = self._s0            
        v[0] = self._v0
        dW1 = standard_normal(self._nSteps)
        dW2 = self._rho * dW1 + (1 - self._rho**2)**(0.5) * standard_normal(self._nSteps)
        for j in range(0, self._nSteps):
            s[j + 1] = s[j] * np.exp((self._r - 0.5 * v[j]) * dt + (v[j] * dt)**(0.5) * dW1[j])
            v[j + 1] = max((self._kappa*(self._theta - v[j])*dt)-(0.25*(self._lamda**2)*dt)+\
                       (v[j]**0.5+0.5*self._lamda*dW2[j]*(dt**(0.5)))**2, 0)
        return s            
    #Pricing with Euler Discretization
    def price(self, option):
        payOff_Sum = 0.0
        for i in range(0, self._nPaths):
            payOff_Sum += option.payoff(self._generate_path(option.T / self._nSteps))
        return (np.exp(- self._r * option.T) * payOff_Sum / self._nPaths)
    #Pricing with Milstein Discretization
    def price_Mil(self, option):
        payOff_Sum = 0.0
        for i in range(0, self._nPaths):
            payOff_Sum += option.payoff(self._generate_path_Mil(option.T / self._nSteps))
        return (np.exp(- self._r * option.T) * payOff_Sum / self._nPaths)
class Option : 
    def __init__(self, K, T):
        self.K = K
        self.T = T
    def payoff(self, underlying_path):
        pass
#Call Option class
class CallOption(Option):
    def payoff(self, underlying_path):
        return max(underlying_path[-1] - self.K, 0)
#Put Option class
class PutOption(Option):
    def payoff(self, underlying_path):
        return max(self.K - underlying_path[-1], 0)
    
import numpy as np
from scipy.stats import norm

#Black Scholes Function
def BS(S, K, T, r, v, callPutFlag = 'c'):
    d1 = (np.log(S / K) + (r + 0.5 * v**2) * T) / (v * np.sqrt(T))
    d2 = d1 - v * np.sqrt(T)
    if (callPutFlag == 'c') or (callPutFlag == 'C'):
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
#Calc Vega    
def BS_vega(S, K, T, r, v):
    d1 = (np.log(S/K)+(r+0.5*v**2)*T)/(v*np.sqrt(T))
    return (S*np.sqrt(T)*norm.cdf(d1))
#Calc Implied volatility using Newton's Method
def IV(price_, S, K, T, r, callPutFlag = 'c'):
    max_it = 200
    pre = 1.0e-5
    v = 0.2
    for i in range(max_it):
        price = BS(S, K, T, r, v, callPutFlag = 'c')
        vega = BS_vega(S, K, T, r, v)
        price = price 
        diff = price_ - price
        if (abs(diff)<pre):
            return v
        v = v + diff/vega
    #return best vol
    return v    

In [8]:
#import numpy as np
import matplotlib.pyplot as plt
#from black_sholes import BS
#from option import CallOption
#from model import HestonModel

#Number of time steps
nSteps = 50
#Number of monte carlo paths
nPaths = 10000
#Strike
#K = 145
#maturity
T = 0.1
#Initial stock price
s0 = 1 #GOOGL
#Initial volatility
v0 = 0.3371
#risk free rate
r = 0
#long term volatility(equiribrium level)
theta = 0
#Mean reversion speed of volatility
kappa = 46.7194
#lambda(volatility of Volatility)
sigma = 0.0491
#rho
rho = 0.059
v0 = v0
rho = rho
sigma = sigma
theta = theta
kappa = kappa

#simulation
K = np.arange(0.5, 1.5, 0.1)
actual = [12.56, 9.55, 7.10, 5.40, 3.90, 2.80, 1.94, 1.41]
heston_ = []
print ("Strike:","Heston:    ", "Actual :   ","Difference:")
i = 0
for k in K:
    call_option = CallOption(k, T)
    heston = HestonModel(nSteps, nPaths, s0, v0, r, theta, kappa, sigma, rho)
    price = heston.price(call_option)
    heston_.append(price)
    print (k,"   ", price, actual[i],"",(price-actual[i])/actual[i])
    i = i+1
    


#plot result
plt.plot(K, heston_,"b")
plt.plot(K, actual,"r")
plt.xlabel('Strike (K)')
plt.ylabel('Option price')
plt.title('JPM')
plt.show()

Strike: Heston:     Actual :    Difference:
0.5     0.5001626096303132 12.56  -0.9601781361759304
0.6     0.4010464224918899 9.55  -0.9580056102102733
0.7     0.29969575702214396 7.1  -0.957789329996881
0.7999999999999999     0.2004310094317926 5.4  -0.96288314640152
0.8999999999999999     0.10288581914006459 3.9  -0.9736190207333167
0.9999999999999999     0.03363279096567038 2.8  -0.9879882889408321
1.0999999999999999     0.005731614079915231 1.94  -0.997045559752621
1.1999999999999997     0.0007026865056726367 1.41  -0.9995016407761186


IndexError: list index out of range

In [14]:
IV(0.40120071417989733,1,0.6,1.1,0,'c')

0.2318993890221547

In [15]:
IV(0.30171049443832915,1,0.7,1.1,0,'c')

0.17881263491448884

In [16]:
IV(0.20104072041111268,1,0.8,1.1,0,'c')

0.11014514331046768

In [17]:
IV(0.1059009501036606,1,0.9,1.1,0,'c')

0.08982815823173457